<a href="https://colab.research.google.com/github/tejaschaudhari2811/FIRE_Project_Task_1/blob/main/FIRE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 : Message Level Sentiment Analysis for Tamil Language

In [2]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 215kB 3.1MB/s 
     |████████████████████████████████| 22.3MB 35.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [17]:
import pandas as pd
import re
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

In [18]:
from google.colab import files
uploaded = files.upload()

In [19]:
df = pd.read_table("tamil_offensive_train.tsv")

In [20]:
df['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [21]:
# Define Polarities for numeric classification
polarities={'NOT':1, "OFF":-1,"not-Tamil":0}

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5880 entries, 0 to 5879
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5880 non-null   object
 1   text      5880 non-null   object
 2   category  5880 non-null   object
dtypes: object(3)
memory usage: 137.9+ KB
None


In [23]:
df.isnull().sum()

id          0
text        0
category    0
dtype: int64

In [24]:
df.groupby(by=['category']).count()

,id,text
category,,
NOT,4724,4724
OFF,1153,1153
not-Tamil,3,3


In [25]:
# Supplementary Functions
def convert_polarity(category):
    return polarities[category]

def word_length(sentence):
    return len(sentence.split(" "))

In [26]:
df['polarity'] = df['category'].apply(convert_polarity)
df['text_length'] = df['text'].apply(len)
df['number_of_words'] = df['text'].apply(word_length)

In [27]:
df.head()

,id,text,category,polarity,text_length,number_of_words
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT,1,60,8
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF,-1,80,12
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT,1,69,7
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF,-1,84,13
4,tam5,only for விஜய் சேதுபதி and STR,NOT,1,30,6


In [28]:
df = df.drop(df[df.polarity == 0].index)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [30]:
tfidf = TfidfVectorizer(max_features=5000)

In [31]:
# Get the data from Dataframe.
X = df['text']
y = df['polarity']

# Convert data to vectors
X = tfidf.fit_transform(X)
X_ros,y_ros = ros.fit_resample(X,y)

In [32]:
# Split The Data
X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2, random_state=0)

In [33]:
# Create an alias for the classifier
clf = LinearSVC()

In [34]:
# Train the classifier.
clf.fit(X_train,y_train)

LinearSVC()

In [35]:
# Get Predictions
y_pred = clf.predict(X_test)

In [36]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.82      0.88      0.85       953
           1       0.87      0.80      0.83       937

    accuracy                           0.84      1890
   macro avg       0.84      0.84      0.84      1890
weighted avg       0.84      0.84      0.84      1890



# Using Logistic Regression for Classification

In [37]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.79      0.83      0.81       953
           1       0.82      0.77      0.80       937

    accuracy                           0.80      1890
   macro avg       0.81      0.80      0.80      1890
weighted avg       0.81      0.80      0.80      1890

